In [4]:
!pip install -U langchain langchain_huggingface python-dotenv langchain chromadb faiss-cpu openai tiktoken langchain_openai langchain-community wikipedia

In [5]:
from langchain_community.retrievers import WikipediaRetriever
from dotenv import load_dotenv
load_dotenv()

False

In [6]:
# Vector Store WikipediaRetriever
wikiRetriever = WikipediaRetriever(top_k_results = 2, lang='en')

In [7]:
result = wikiRetriever.invoke("Ipl 2025 Winner list")

In [8]:
for index, doc in enumerate(result):
  print(f"content {index + 1}")
  print(doc.page_content)

content 1
The 2025 Indian Premier League, also known as IPL 18 and branded as TATA IPL 2025, was the 18th edition of the Indian Premier League, a professional Twenty20 cricket league. The tournament featured 10 teams competing in 74 matches. It began on 22 March and was held across 13 venues before being suspended on 9 May due to the 2025 India–Pakistan crisis. The matches resumed from 17 May across six venues, with the final rescheduled from 25 May to 3 June.
The defending champions Kolkata Knight Riders were eliminated in the league stage. In the final, Royal Challengers Bengaluru defeated Punjab Kings by 6 runs to win their maiden title after 18 years. Sai Sudharsan and Prasidh Krishna of Gujarat Titans scored the most runs (759) and took the most wickets (25) respectively. Suryakumar Yadav of Mumbai Indians was awarded as the most valuable player of the tournament.


== Background ==
The Indian Premier League (IPL) is a professional Twenty20 (T20) cricket league held in India, orga

In [9]:
# Vector Store Retriever
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings, ChatHuggingFace
from langchain_core.documents import Document

In [10]:
embedding  = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [12]:
vector_store = Chroma.from_documents(
    documents = documents,
    embedding = embedding,
    collection_name = 'my_collecions',
    persist_directory='/content/drive/MyDrive/ML-AI/GenAI/LangChain/code-base/retrievers/my_chroma'
)

In [13]:
vectorRetriever = vector_store.as_retriever(search_kwargs={'k' : 2})

In [14]:
docs = vectorRetriever.invoke("What is Chroma used for?")
for i, doc in enumerate(docs):
  print(f"content {i + 1}")
  print(doc)

content 1
page_content='Chroma is a vector database optimized for LLM-based search.'
content 2
page_content='Chroma is a vector database optimized for LLM-based search.'


In [15]:
result = vector_store.similarity_search("what us chroma sued for ?", k = 2)
for i, doc in enumerate(result):
  print(f"content {i + 1}")
  print(doc)
  # both doing same things but retreiver can use other search strategy

content 1
page_content='Chroma is a vector database optimized for LLM-based search.'
content 2
page_content='Chroma is a vector database optimized for LLM-based search.'


In [ ]:
#MMR (Minimal Marginal Retriever)

In [16]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [18]:

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEndpoint
#from langchain.retrievers.multi_query import MultiQueryRetriever


In [ ]:
vector_Faiss = FAISS.from_documents(documents=docs,
                                    embedding = embedding)


In [ ]:
mmrReteriver = vector_Faiss.as_retriever(    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0}  # k = top results, lambda_mult = relevance-diversity balance)
)

In [ ]:
result = mmrReteriver.invoke('what is langchain?')
for i, doc in enumerate(result):
  print(f"content {i+1}")
  print(doc)

content 1
page_content='LangChain supports Chroma, FAISS, Pinecone, and more.'
content 2
page_content='MMR helps you get diverse results when doing similarity search.'
content 3
page_content='Embeddings are vector representations of text.'


In [ ]:
# multi query retrivers

In [ ]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [ ]:
vector_Faiss = FAISS.from_documents(documents=all_docs, embedding=embedding)


In [ ]:
similarity_retriever = vector_Faiss.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
llm = HuggingFaceEndpoint(repo_id='HuggingFaceH4/zephyr-7b-beta',
                           task="text-generation",
                           max_new_tokens = 50)
chat_model = ChatHuggingFace(llm=llm)
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vector_Faiss.as_retriever(search_kwargs={"k": 5}),
    llm=chat_model
)

# Query
query = "How to improve energy levels and maintain balance?"

# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [ ]:
# Contextual Compression Retriver